In [1]:
import sys
sys.path.append('..')
from chemex.web import cs_scrape_properties
from itertools import islice
import pandas as pd
from pandas import DataFrame

In [76]:
csid = 4471
one_data = cs_scrape_properties(csid)

In [77]:
one_data.keys()

dict_keys(['Experimental Melting Point', 'Experimental LogP', 'Compound Source', 'More details', 'Bio Activity', 'Retention Index (Linear)', 'EPI Suite', 'Predicted Melting Point', 'Retention Index (Kovats)', 'Experimental Flash Point', 'Safety', 'Experimental Boiling Point', 'Experimental Gravity'])

In [8]:
props_of_interest = ['Experimental Melting Point',
                     'Experimental Boiling Point',
                     'Experimental Vapor Pressure',
                     'Experimental Solubility',
                     'Experimental LogP',
                     'Experimental Vapor Pressure']

In [9]:
csid_list = [4471, 5889, 8677, 20939]

In [82]:
def cs_props_multi(csids, props=props_of_interest):
    data = dict()
    for csid in csids:
        d = cs_scrape_properties(csid)
        data[csid] = {k: v for (k, v) in d.items() if k in props}
    return data

In [83]:
multi_data = cs_props_multi(csid_list)

In [84]:
DataFrame(multi_data).transpose()

,Experimental Boiling Point,Experimental LogP,Experimental Melting Point,Experimental Solubility,Experimental Vapor Pressure
4471,[150 deg C / 5 mm (346.3046 °C / 760 mmHg)\r\n...,"[3.641 Vitas-M STK057962, 2.9758 Synthon-Lab ...","[63 °C TCI H0266, 62-65 °C Alfa Aesar, 64 °C J...",NaN,NaN
5889,"[183-184 °C Alfa Aesar, 363 F (183.8889 °C)\r\...",[0.9 Egon Willighagen http://dx.doi.org/10.102...,"[-6 °C Alfa Aesar, -6 °C Oxford University Che...","[4% NIOSH BW6650000, Soluble in water Alfa Aes...",[0.6 mmHg NIOSH BW6650000]
8677,NaN,NaN,"[206 °C Alfa Aesar, 204-207 °C Oxford Universi...",NaN,NaN
20939,"[170-180 °C Alfa Aesar, 175-177 °C Food and Ag...",NaN,"[-40 °C LKT Labs \r\n [L3250], -74...",[Insoluble in water. LKT Labs \r\n ...,NaN


In [85]:
for x in one_data['EPI Suite'].split('\n'):
    print(x)



Predicted data is generated using the US Environmental Protection Agency’s EPISuite

                        
 Log Octanol-Water Partition Coef (SRC):
    Log Kow (KOWWIN v1.67 estimate) =  3.52
    Log Kow (Exper. database match) =  3.79
       Exper. Ref:  Chem Inspect Test Inst (1992)

 Boiling Pt, Melting Pt, Vapor Pressure Estimations (MPBPWIN v1.42):
    Boiling Pt (deg C):  363.41  (Adapted Stein & Brown method)
    Melting Pt (deg C):  129.80  (Mean or Weighted MP)
    VP(mm Hg,25 deg C):  6.62E-006  (Modified Grain method)
    MP  (exp database):  65.5 deg C
    BP  (exp database):  155 @ 5 mm Hg deg C
    Subcooled liquid VP: 1.59E-005 mm Hg (25 deg C, Mod-Grain method)

 Water Solubility Estimate from Log Kow (WSKOW v1.41):
    Water Solubility at 25 deg C (mg/L):  68.56
       log Kow used: 3.79 (expkow database)
       no-melting pt equation used

 Water Sol Estimate from Fragments:
    Wat Sol (v1.01 est) =  2295.4 mg/L

 ECOSAR Class Program (ECOSAR v0.99h):
    Class

In [138]:
def dict_from_line(txt, sep=':'):
    d = [s.strip() for s in txt.split(sep, 1)]
    return {d[0]: d[1]}

def epi_suite_values(epi_str):
    data = dict()
    lines = epi_str.split('\n')
    for i in lines:
        j = i.strip()
        if j.startswith('Log Kow (') or j.startswith('Log BCF'):
            data.update(dict_from_line(i, '='))
        if j.startswith('Henrys LC') or\
            j.startswith('Log Koa (KOAWIN') or\
            j.startswith('Log Koa (experimental') or\
            j.startswith('Persistence Time') or\
            j.startswith('Ready Biodegradability Prediction'):
            data.update(dict_from_line(i))
    return data

d = epi_suite_values(one_data['EPI Suite'])
for x in sorted(d.keys()):
    print(x, d[x], sep=': ')

Henrys LC [VP/WSol estimate using EPI values]: 2.900E-008 atm-m3/mole
Log BCF from regression-based method: 1.378 (BCF = 23.89)
Log Koa (KOAWIN v1.10 estimate): 10.002
Log Koa (experimental database): None
Log Kow (Exper. database match): 3.79
Log Kow (KOWWIN v1.67 estimate): 3.52
Persistence Time: 1.26e+003 hr
Ready Biodegradability Prediction: NO
